## Función get_all_tweets (Adaptada)
##### Adaptación de la función get_all_tweets (seankross) que elimina el problema de la extensión, incluye más variables y a su vez se incluye la versión iterada

Versión original de la función: https://gist.github.com/seankross/9338551

In [ ]:
import tweepy 
import csv
import pandas as pd
import os
import glob

In [ ]:
#Twitter API credentials
consumer_key = ''
consumer_secret = ''
access_key = ''
access_secret = ''

In [ ]:
def get_all_tweets_adap(screen_name):
    #Twitter solo permite extracción de 3240 trinos con este método
    
    #Datos de Twitter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #Guardar todos los datos
    alltweets = []  
    
    #Hacer el request
    assert (screen_name in ["nan", None]), "Casilla en Blanco"
    
    new_tweets = api.user_timeline(screen_name = screen_name,count=200, tweet_mode="extended")
    
    #Guardar 
    alltweets.extend(new_tweets)
    
    #Guardar los datos
    oldest = alltweets[-1].id - 1
    
    #Seguir recogiendo datos
    while len(new_tweets) > 0:
        print(f"getting tweets from {screen_name}")
        
        
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest, tweet_mode='extended')
        
        #Guardar
        alltweets.extend(new_tweets)
        
        #Guardar
        oldest = alltweets[-1].id - 1
        
        print(f"...{len(alltweets)} tweets downloaded so far")
    
    #Transformar las listas a un dataframe
    outtweets=[]
    for tweet in alltweets:
        if tweet.full_text.startswith("RT @") == True:
            try:
                outtweets.append([tweet.user.screen_name, tweet.created_at, tweet.retweeted_status.full_text,tweet.user.location, tweet.user.description,tweet.retweet_count,tweet.favorite_count,tweet.in_reply_to_screen_name,tweet.coordinates,tweet.place,True, tweet.retweeted_status.user.screen_name])
            except:
                outtweets.append([tweet.user.screen_name, tweet.created_at, tweet.full_text,tweet.user.location, tweet.user.description,tweet.retweet_count,tweet.favorite_count,tweet.in_reply_to_screen_name,tweet.coordinates,tweet.place,False,None])
        else:
            outtweets.append([tweet.user.screen_name, tweet.created_at, tweet.full_text,tweet.user.location, tweet.user.description,tweet.retweet_count,tweet.favorite_count,tweet.in_reply_to_screen_name,tweet.coordinates,tweet.place,False,None])


    #Guardar el csv  
    with open(f'new_{screen_name}_tweets.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text","location","bio","retweets","favourites","in_reply_to_screen_name", "coordinates","place", "is_retweet", "retweeted_account_id"])
        writer.writerows(outtweets)
    
    pass

In [ ]:
get_all_tweets_adap("") #Una sola cuenta

In [ ]:
#Establecer un directorio
os.chdir(r"")

In [ ]:
#Para varias cuentas
for i in nombres:
    try:
        get_all_tweets_adap(i)
    except:
        pass

In [ ]:
#Agrupar todos los archivos en uno solo parte 1
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
#Agrupar todos los archivos en uno solo parte 2
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

In [ ]:
#Agrupar todos los archivos en uno solo parte 3
combined_csv.to_excel( "combined.xlsx", index=False, encoding='utf-8-sig')